In [ ]:
#!pip install --upgrade pip
!pip install --upgrade "pip<24.1"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
#!pip install fairseq #for finetuning\
!pip install --upgrade datasets
!pip install transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is i

In [2]:
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [3]:
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor
import json

# Create a tokenizer
#tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/Colab Notebooks/FYP Research Artefacts/vocab.json", unk_token="<unk>", pad_token="<pad>", word_delimiter_token="|")

# Create a feature extractor
#feature_extractor = Wav2Vec2FeatureExtractor() # Create a default feature extractor

# Create a processor (combining feature extractor & tokenizer)
#processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")

# Save processor to disk
#processor.save_pretrained("custom_processor")

In [9]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torchaudio
import torch

# Load the Wav2Vec2 model (pre-trained but not fine-tuned)
model_name = "facebook/wav2vec2-large-960h"
model = Wav2Vec2ForCTC.from_pretrained(model_name)


# Load an audio file
waveform, sample_rate = torchaudio.load("/content/drive/MyDrive/Colab Notebooks/FYP Research Artefacts/proof of concept/3.wav")

# Convert to 16kHz (Wav2Vec2 requires this)
if sample_rate != 16000:
    waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)

# Process the audio
input_values = processor(waveform.squeeze().numpy(), return_tensors="pt", padding=True, sampling_rate=16000).input_values

# Get predictions
with torch.no_grad():
    logits = model(input_values).logits

# Convert predictions to text
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

print(processor.tokenizer.get_vocab())


print("Transcription:", transcription[0])  # Output the text


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3, '|': 4, 'E': 5, 'T': 6, 'A': 7, 'O': 8, 'N': 9, 'I': 10, 'H': 11, 'S': 12, 'R': 13, 'D': 14, 'L': 15, 'U': 16, 'M': 17, 'W': 18, 'C': 19, 'F': 20, 'G': 21, 'Y': 22, 'P': 23, 'B': 24, 'V': 25, 'K': 26, "'": 27, 'X': 28, 'J': 29, 'Q': 30, 'Z': 31}
Transcription: I NEVER REALLY ASK SHE BUT SHIS SITUATION BECUT WHAN I DEALING WITH A GIRL I DON'T CARE BOT SHE PART HAD ON REALLY KEBOLD ON NOW YOU KNOW WHEN I DEALING WITH A GIRL IS ME SHE NOMONEY IN GON NOR NOBONEY BLIKES TO KNOW YOU KNOW WE COULD KEEP IT LOW KEY YOU KNOW THIS ME FROM BUT MEAN AL THIS MONEY BE EVEN S ON AG I TELL YOU JUST SO MY TEXT MEN TWELVE O'CROCKIN IN IT TWE O'CLOCK IN THE MORNING BUT BOR SHIFE IF I WANT HE SENT PITY WIT HIS HE IN AAKME THAT ONER IS TO GRIN A FOR THIS LITTLE MARBY WILL TALK I DON'T TELL HIM ONTAT I JUST LEFT AS THOUGH


# Load the dataset

In [4]:
!git clone https://github.com/kchase9/creolese-audio-dataset.git

Cloning into 'creolese-audio-dataset'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 90 (delta 8), reused 15 (delta 5), pack-reused 70 (from 2)
Receiving objects: 100% (90/90), 455.70 MiB | 30.27 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Updating files: 100% (71/71), done.


In [5]:
audio_path = "/content/creolese-audio-dataset/Audio Files"
transcription_path = "/content/creolese-audio-dataset/transcripts.json"

# Start compiling the references and hypotheses

In [8]:
pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.9 MB/s eta 0:00:00


In [7]:
import os
import json
import torchaudio
import torch
from jiwer import wer, cer, mer

In [ ]:
ground_truth = []
predictions= []

with open(transcription_path, "r") as f:
    dataset = json.load(f)

for item in dataset:
  filename= ""
  audio_path = "/content/creolese-audio-dataset/Audio Files"
  filename = os.path.basename(item["audio"])
  audio_path = os.path.join(audio_path, filename)
  reference = item["text"]
  try:
      waveform, sample_rate = torchaudio.load(audio_path)
      input_values = processor(waveform.squeeze().numpy(), return_tensors="pt", padding=True, sampling_rate=16000).input_values
      logits = model(input_values).logits
      predicted_ids = torch.argmax(logits, dim=-1)
      transcription = processor.batch_decode(predicted_ids)
      hypothesis = transcription[0]

  except Exception as e:
      print(f"Skipping {audio_path}: {e}")
      continue

ground_truth.append(reference)
predictions.append(hypothesis)

In [ ]:
print(ground_truth)
print(predictions)